In [7]:
import math
import sys
import os
import cv2
import numpy as np
import random
from matplotlib import pyplot as plt
from IPython.display import clear_output
%matplotlib inline

In [13]:
def sssd(frame, next_frame, grid1, grid2):
    cur_blk_r = frame[grid1[0]-2:grid1[0]+3, grid1[1]-2:grid1[1]+3, 0]
    cur_blk_g = frame[grid1[0]-2:grid1[0]+3, grid1[1]-2:grid1[1]+3, 1]
    cur_blk_b = frame[grid1[0]-2:grid1[0]+3, grid1[1]-2:grid1[1]+3, 2]
    
    tar_blk_r = next_frame[grid2[0]-2:grid2[0]+3, grid2[1]-2:grid2[1]+3, 0]
    tar_blk_g = next_frame[grid2[0]-2:grid2[0]+3, grid2[1]-2:grid2[1]+3, 1]
    tar_blk_b = next_frame[grid2[0]-2:grid2[0]+3, grid2[1]-2:grid2[1]+3, 2]

    diff_r = np.subtract(cur_blk_r, tar_blk_r)
    diff_g = np.subtract(cur_blk_g, tar_blk_g)
    diff_b = np.subtract(cur_blk_b, tar_blk_b)
    
    sqr_r = np.square(diff_r)
    sqr_g = np.square(diff_g)
    sqr_b = np.square(diff_b)
    
    sqr_sum = sqr_r + sqr_g + sqr_b
    sqr_sum = np.sum(sqr_sum)
    dist = math.sqrt(sqr_sum) 
    return dist

def min_sssd(frame, next_frame, grid):
    cur_min = [0, 0, 0]
    if sssd(frame, next_frame, grid, grid) <= tmin:
        return cur_min
    for i in range(-1, 2):
        for j in range(-1, 2):
            if (i == 0 and j == 0) or grid[0] + i <= 1 or grid[0] + i >= frame_height - 2 or grid[1] + j <= 1 or grid[1] + j >= frame_width - 2:
                continue
            test_grid = [grid[0] + i, grid[1] + j]
            temp = sssd(frame, next_frame, grid, test_grid)
            if (temp < cur_min[0] or cur_min[0] == 0):
                cur_min = [temp, grid[0] + i, grid[1] + j]
    if cur_min[0] < tmin:
        return [0, 0, 0]
    for i in range(-1, 2):
        for j in range(-1, 2):
            if (i == 0 and j == 0) or ((cur_min[1] - grid[0]) + (cur_min[2] - grid[1])) < 2 or cur_min[1] + i <= 1 or cur_min[1] + i >= frame_height - 2 or cur_min[2] + j <= 1 or cur_min[2] + j >= frame_width - 2:
                continue
            test_grid = [cur_min[1] + i, cur_min[2] + j]
            temp = sssd(frame, next_frame, grid, test_grid)
            if (temp < cur_min[0] or cur_min[0] == 0):
                cur_min = [temp, grid[0] + i, grid[1] + j]
    return cur_min
                

In [9]:
def arrowdraw(img, x1, y1, x2, y2):
    radians = math.atan2(x1-x2, y2-y1)
    x11 = 0
    y11 = 0
    x12 = -10
    y12 = -10

    u11 = 0
    v11 = 0
    u12 = 10
    v12 = -10
    
    x11_ = x11*math.cos(radians) - y11*math.sin(radians) + x2
    y11_ = x11*math.sin(radians) + y11*math.cos(radians) + y2

    x12_ = x12 * math.cos(radians) - y12 * math.sin(radians) + x2
    y12_ = x12 * math.sin(radians) + y12 * math.cos(radians) + y2
    
    u11_ = u11 * math.cos(radians) - v11 * math.sin(radians) + x2
    v11_ = u11 * math.sin(radians) + v11 * math.cos(radians) + y2

    u12_ = u12 * math.cos(radians) - v12 * math.sin(radians) + x2
    v12_ = u12 * math.sin(radians) + v12 * math.cos(radians) + y2
    img = cv2.line(img, (x1, y1), (x2, y2), (255, 255, 255), 1)
    img = cv2.line(img, (int(x11_), int(y11_)), (int(x12_), int(y12_)), 
    (255, 255, 255), 1)
    img = cv2.line(img, (int(u11_), int(v11_)), (int(u12_), int(v12_)), 
    (255, 255, 255), 1)
    
    return img

In [10]:
if not os.path.isdir(os.path.join(os.getcwd(), 'frames')):
    os.mkdir("frames")
else:
    print('frames already exists')

if not os.path.isdir(os.path.join(os.getcwd(), 'motion')):
    os.mkdir("motion")
else:
    print('motion already exists')
    
framenumber = 0
framectr = 0
omovie = cv2.VideoCapture('original.avi')
frame_height = omovie.get(cv2.CAP_PROP_FRAME_HEIGHT)
frame_width  = omovie.get(cv2.CAP_PROP_FRAME_WIDTH)

while(1):
    ret, frame = omovie.read()
    if not ret:
        break
    print('Extracting: %d' % framenumber)
    clear_output(wait=True)
    cv2.imwrite('frames/%d.tif' % framenumber, frame)
    framenumber += 1
omovie.release()

Extracting: 751


In [14]:
framectr = framenumber - 1
process_frame = 0
tmin = 55
tmax = 65
grid_row = int(frame_height/5)
grid_column = int(frame_width/5)
grid = np.zeros([grid_row, grid_column, 2], dtype = int)
for i in range(grid_row):
    for j in range(grid_column):
        grid[i][j] = [5*i+2, 5*j+2]
while process_frame < framectr:

    frame = cv2.imread('frames/%d.tif' % process_frame)
    next_frame = cv2.imread('frames/%d.tif' % (process_frame + 1))
    print('Processing frame: %d, overall progress: %.2f %%' % (process_frame, process_frame/framectr*100))
    clear_output(wait=True)

    
    for i in range(grid_row):
        for j in range(grid_column):
            matched_grid = min_sssd(frame, next_frame, grid[i][j])
            
            if matched_grid[0] != 0 and matched_grid[0] >= tmin and matched_grid[0] <=tmax:
                frame = arrowdraw(frame, grid[i][j][1], grid[i][j][0], int(matched_grid[2]), int(matched_grid[1]))
        
    
    cv2.imwrite('motion/motion%d.tif' % process_frame, frame)
    if cv2.waitKey(30) & 0xff == ord('q'):
        break
    process_frame += 1

Processing frame: 750, overall progress: 99.87 %


In [15]:
count = 0
out = cv2.VideoWriter('./motion_estimation.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (int(frame_width), int(frame_height)))
while(1):
    img = cv2.imread('motion/motion%d.tif' % count)
    if img is None:
        print('No more frames to be loaded')
        break
    clear_output(wait=True)
    out.write(img)
    count += 1
    print('Saving video: %d%%' % int(100*count/framenumber))
    
out.release()
cv2.destroyAllWindows()

Saving video: 99%
No more frames to be loaded
